#### Prepared for Gabor's Data Analysis

### Data Analysis for Business, Economics, and Policy
by Gabor Bekes and  Gabor Kezdi
 
Cambridge University Press 2021

**[gabors-data-analysis.com ](https://gabors-data-analysis.com/)**

 License: Free to share, modify and use for educational purposes. 
 Not to be used for commercial purposes.


### CHAPTER 20

**CH20B Fine tuning social media advertising**

using the ab-test-social-media dataset

version 1.0 2021-05-05

In [1]:
import os
import sys
import warnings

import pandas as pd
import pyfixest as pf
import statsmodels.stats.api as sms
from statsmodels.stats.power import TTestIndPower

warnings.filterwarnings("ignore")


In [2]:
# Current script folder
current_path = os.getcwd()
dirname = current_path.split("da_case_studies")[0]

# location folders
data_in = dirname + "da_data_repo/ab-test-social-media/clean"
data_out = dirname + "da_case_studies/ch20-ab-test-social-media/"
output = dirname + "da_case_studies/ch20-ab-test-social-media/output/"

func = dirname + "da_case_studies/ch00-tech-prep/"
sys.path.append(func)


## Part I

sample size calculation

sample size calculation with planned rates

In [3]:
clickthrough = 0.01

conversion = 0.05

proportionA = clickthrough * conversion

proportionB = proportionA * 1.2

es = sms.proportion_effectsize(proportionA, proportionB)
# es = proportionB - proportionA
TTestIndPower().solve_power(es, power=0.8, alpha=0.05) * 2


1722229.753552455

In [4]:
clickthrough = 0.0032

conversion = 0.0082

proportionA = clickthrough * conversion

proportionB = proportionA * 1.2

es = sms.proportion_effectsize(proportionA, proportionB)
# es = proportionB - proportionA
TTestIndPower().solve_power(es, power=0.8, alpha=0.05) * 2


32833931.071609996

## Part II

p-value of tests

In [5]:
summary_data = pd.read_excel(data_in + "/ab-test-summary.xlsx").set_index("action_type")
# summary_data = pd.read_excel("https://osf.io/download/mhybr/").set_index("action_type")


In [6]:
summary_data


,show,clicks,action
action_type,,,
Action A,1000000,3323,32
Action B,1000000,3128,21


In [7]:
data = pd.DataFrame(0, columns=["type_b", "clicks", "action"], index=range(0, 2000000))

data.loc[0:999999, "type_b"] = 1

data.loc[0 : summary_data.loc["Action A", "clicks"] - 1, "clicks"] = 1
data.loc[1000000 : 1000000 + summary_data.loc["Action B", "clicks"] - 1, "clicks"] = 1
data.loc[0 : summary_data.loc["Action A", "action"] - 1, "action"] = 1
data.loc[1000000 : 1000000 + summary_data.loc["Action B", "action"] - 1, "action"] = 1


In [8]:
data.groupby(["type_b", "clicks"]).count()


action
type_b clicks        
0      0       996872
       1         3128
1      0       996677
       1         3323

In [9]:
data.groupby(["type_b", "action"]).count()


clicks
type_b action        
0      0       999979
       1           21
1      0       999968
       1           32

In [11]:
pf.feols("clicks ~ 1 + type_b", data, vcov="hetero").summary()

###

Estimation:  OLS
Dep. var.: clicks, Fixed effects: 0
Inference:  hetero
Observations:  2000000

| Coefficient   |   Estimate |   Std. Error |   t value |   Pr(>|t|) |   2.5% |   97.5% |
|:--------------|-----------:|-------------:|----------:|-----------:|-------:|--------:|
| Intercept     |      0.003 |        0.000 |    56.016 |      0.000 |  0.003 |   0.003 |
| type_b        |      0.000 |        0.000 |     2.432 |      0.015 |  0.000 |   0.000 |
---
RMSE: 0.057 R2: 0.0 


In [12]:
pf.feols("action ~ 1 + type_b", data, vcov="hetero").summary()

###

Estimation:  OLS
Dep. var.: action, Fixed effects: 0
Inference:  hetero
Observations:  2000000

| Coefficient   |   Estimate |   Std. Error |   t value |   Pr(>|t|) |   2.5% |   97.5% |
|:--------------|-----------:|-------------:|----------:|-----------:|-------:|--------:|
| Intercept     |      0.000 |        0.000 |     4.583 |      0.000 |  0.000 |   0.000 |
| type_b        |      0.000 |        0.000 |     1.511 |      0.131 | -0.000 |   0.000 |
---
RMSE: 0.005 R2: 0.0 
